# Interactive Face Detection Sample

This sample showcases Object Detection task applied for face recognition using sequence of neural networks.
Async API usage can improve overall frame-rate of the application, because rather than wait for inference to complete,
the application can continue operating on the host while accelerator is busy.
This sample maintains three parallel infer requests for the Age/Gender Recognition, Head Pose Estimation, and Emotions Recognition that run simultaneously.

Other sample objectives are:

*	Video as input support via OpenCV
*	Visualization of the resulting face bounding boxes from Face Detection network
*	Visualization of age/gender, head pose and emotion information for each detected face

OpenCV is used to draw resulting bounding boxes, labels, and other information. You can copy and paste this code without pulling Inference Engine sample helpers into your application

## How it Works

*	The application reads command line parameters and loads up to four networks depending on `-m...` options family to the Inference Engine.
*	The application gets a frame from the OpenCV's VideoCapture.
*	The application performs inference on the frame detection network.
*	The application performs three simultaneous inferences, using the Age/Gender, Head Pose and Emotions detection networks if they are specified in command line.
*	The application displays the results.

The new Async API operates with a new notion of the Infer Request that encapsulates the inputs/outputs and separates scheduling and waiting for result. For more information about Async API and the difference between Sync and Async modes performance, refer to **How it Works** and **Async API** sections in [Object Detection SSD, Async API Performance Showcase Sample](@ref InferenceEngineObjectDetectionSSDDemoAsyncApplication).


## Compiling

There are two source files: 
* [detectors.cpp](detectors.cpp) -- contains the various helper functions
* [main.cpp](main.cpp) -- contains the main loop

We have provided a Makefile for compiling the application. Run the following cell to run the make command.

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *

In [ ]:
%%bash
source /opt/intel/openvino/bin/setupvars.sh

In [ ]:
!make

This produces an executable [interactive_face_detection_demo](). This executable takes in a number of different command line arguments.

Run the following cell to see the list: 

In [ ]:
%%bash
./interactive_face_detection_demo -h

The version of the cpp file here is a slightly modified version of the faca_detector code built-in to the the Intel® Distribution of OpenVINO™ toolkit.
In this version, the result is written into a output mp4 file specified by the `-o` flag. 

### Downloading IR Model

The Intel® Distribution of OpenVINO™ toolkit comes with model downloader that can download pre-compiled Intermediate Representation (IR) models that are optimized for different end-point target devices.
These models can be created from existing DNN models from popular frameworks (e.g. Caffe*, TF) using the Model Optimizer. 

Run the following cell to see the models available through `model_downloader.py`

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --print_all

**Note** the '!' is a special Jupyter Notebook command that allows you to run shell commands as if you are in a commannd line. So the above command will work straight out of the box on in a terminal (with '!' removed).

In this demo we will be using the 4 different types of models, with two model files each for FP16 and FP32. These models can be downloaded with the following command.

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name face-detection-adas-0001 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name head-pose-estimation-adas-0001 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name age-gender-recognition-retail-0013 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name emotions-recognition-retail-0003 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name facial-landmarks-35-adas-0002 -o models

The input arguments are as follows:
* --name : name of the model you want to download. It should be one of the models listed in the previous cell
* -o : output directory. If this directory does not exist, it will be created for you.

There are more arguments to this script and you can get the full list using the `-h` option.


### Running the inference

Now we are ready to run the inference workload. In this step we will be submitting the workload as a job to the job queue.

Currently, you are on what is called a "devnode". On this system, you are allocated just one core on a large Intel® Xeon® CPU. The purpose of this node is to develop code on the devnode and run minimal sections of Jupyter* Notebooks, but it is not meant for compute intensive jobs like deep learning inference. So we need to request additional resources from the cluster of Edge nodes to run the inference, and this is done through the job queue.

To put an item on the job queue, we must first create a bash script that run the workload we want. Run the following cell to create bash script [face_detection_demo.sh](face_detection_demo.sh) which will be our job script. 

In [ ]:
%%writefile face_detection_demo.sh
#PBS
INPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3
OUTPUT_FILE=$4

mkdir -p $4

if [ "$DEVICE" = "HETERO:FPGA,CPU" ]; then
    # Environment variables and compilation for edge compute nodes with FPGAs
    source /opt/fpga_support_files/setup_env.sh
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_sg1_bitstreams/2019R3_PV_PL1_FP16_MobileNet_Clamp.aocx
fi

#if [ "$FP_MODEL" = "FP16" ]; then
#  FPEXT='-fp16'
#fi

cd $PBS_O_WORKDIR
export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/
    
./interactive_face_detection_demo -i $INPUT_FILE \
-m models/intel/face-detection-adas-0001/$FP_MODEL/face-detection-adas-0001.xml \
-m_hp models/intel/head-pose-estimation-adas-0001/$FP_MODEL/head-pose-estimation-adas-0001.xml \
-m_ag models/intel/age-gender-recognition-retail-0013/$FP_MODEL/age-gender-recognition-retail-0013.xml \
-m_em models/intel/emotions-recognition-retail-0003/$FP_MODEL/emotions-recognition-retail-0003.xml \
-m_lm models/intel/facial-landmarks-35-adas-0002/$FP_MODEL/facial-landmarks-35-adas-0002.xml \
-l /opt/intel/openvino/inference_engine/lib/intel64/libcpu_extension_sse4.so \
-d $DEVICE \
-o $OUTPUT_FILE

To put this script on the job queue, we use the command `qsub`.
There are two important arguments we use with this command.

First, the `-l` flag.
This flag is used to specify what type of resources to request from the cluster.
For example this can be used to request an Intel® Xeon® CPU based system, or it can be used to request a system with an FPGA accelerator card in it.
The syntax is `-l nodes=1:<tag>` where `<tag>` is the descriptor tag for the resource you want.
For example, `-l nodes=1:tank-870:e3-1268l-v5` will request an Intel® Xeon® system.
To see the list of available tags, and the number of avilable systems, run the following cell.

In [ ]:
!pbsnodes | grep compnode | sort | uniq -c

Then there is the `-F` flag, which is used to pass in arguments to the job script.
The [face_detection_demo.sh](face_detection_demo.sh) takes in 2 arguments:
1) the path to the video to run inference on
2) targeted device (CPU,GPU,MYRIAD)
The job scheduler will use the contents of `-F` flag as the argument to the job script.

The following line will request an Intel Xeon system, and passes in "faces-recognition-walking.mp4 CPU" to the job script. Run the cell to submit this job. 

In [ ]:
os.environ["VIDEO"] = "faces-recognition-walking-and-pause.mp4"

#### Submitting to an edge compute node with an Intel® Core™ CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel 
    Core i5-6500TE</a>. The inference workload will run on the CPU.

In [ ]:
print("Submitting job to Intel Core CPU...")
#Submit job to the queue
job_id_core = !qsub face_detection_demo.sh -l nodes=1:idc001skl:tank-870:i5-6500te -F "$VIDEO CPU FP32 results/core" -N face_detection_core
print(job_id_core[0])

#Progress indicators
if job_id_core:
    progressIndicator('results/core', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)

#### Submitting to an edge compute node with Intel® Xeon® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz-">Intel 
    Xeon Processor E3-1268L v5</a>. The inference workload will run on the CPU.

In [ ]:
print("Submitting job to Intel Xeon CPU...")
#Submit job to the queue
job_id_xeon = !qsub face_detection_demo.sh -l nodes=1:idc007xv5:e3-1268l-v5 -F "$VIDEO CPU FP32 results/xeon" -N face_detection_xeon
print(job_id_xeon[0])

#Progress indicators
if job_id_xeon:
    progressIndicator('results/xeon', 'i_progress_'+job_id_xeon[0]+'.txt', "Inference", 0, 100)

Here you should see a output like "{job_id}.c003", where {job_id} is a number.
This is your job ID, and this value can be used to check on the progress of the job down the line.
Furthermore, the above job script has been written so that it uses its job_id as the name of the output video.   

One bigadvantage of the Job queue system is that you may submit multiple jobs at once. 
These jobs will be run as soon as resources are available, and may all run at once if the cluster is not busy.
Here are few other "preset" jobs that for your convenience that you can run. 

#### Submitting to an edge compute node with Intel® Core™ CPU and using the onboard Intel GPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500TE</a>. The inference workload will run on the Intel® HD Graphics 530 card integrated with the CPU.

In [ ]:
print("Submitting job to Intel Core CPU with Intel GPU...")
#Submit job to the queue
job_id_gpu = !qsub face_detection_demo.sh -l nodes=1:idc001skl:intel-hd-530 -F "$VIDEO GPU FP32 results/gpu" -N face_detection_gpu
print(job_id_gpu[0])

#Progress indicators
if job_id_gpu:
    progressIndicator('results/gpu', 'i_progress_'+job_id_gpu[0]+'.txt', "Inference", 0, 100)

#### Submitting to an edge compute node with  IEI Mustang-F100-A10 (Intel® Arria® 10 FPGA)
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500te CPU</a> . The inference workload will run on the <a href="https://www.ieiworld.com/mustang-f100/en/"> IEI Mustang-F100-A10 </a> card installed in this node.

In [ ]:
print("Submitting job to node with Intel FPGA HDDL-F...")
#Submit job to the queue
job_id_fpga = !qsub face_detection_demo.sh -l nodes=1:idc003a10:iei-mustang-f100-a10 -F "$VIDEO HETERO:FPGA,CPU FP32 results/fpga"
print(job_id_fpga[0])

#Progress indicators
if job_id_fpga:
    progressIndicator('results/fpga', 'i_progress_'+job_id_fpga[0]+'.txt', "Inference", 0, 100)

#### Submitting to an edge compute node with Intel® Movidius™ Neural Compute Stick 2
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500te CPU</a>. The inference workload will run on an <a 
    href="https://software.intel.com/en-us/neural-compute-stick">Intel Neural Compute Stick 2</a> installed in this  node.

In [ ]:
print("Submitting job to node with Intel NCS2...")
#Submit job to the queue
job_id_ncs2 = !qsub face_detection_demo.sh -l nodes=1:idc004nc2:intel-ncs2 -F "$VIDEO MYRIAD FP16 results/ncs2"
print(job_id_ncs2[0])

#Progress indicators
if job_id_ncs2:
    progressIndicator('results/ncs2', 'i_progress_'+job_id_ncs2[0]+'.txt', "Inference", 0, 100)

The `qstat` command is used to track the progress of the jobs. 
We have provided a utility funtion "liveQstat()" that provide a live updating GUI for you.
Run the following cell to check on the progress of your earlier jobs.

In [ ]:
liveQstat()

Once the jobs are done. The following cell can be used to display the output. 
The videoHTML is a utility function provided in [demoutils.py](demoutils.py).
This takes one argument, which is the path to the video (see above for convention for the path name).

For your convenience we have stored the jobid from the above `qsub` commands, and the following cells 

### View Results

In [ ]:
videoHTML('IEI Tank (Intel Core CPU)', 
          ['results/core/output.mp4'],
          'results/core/stats_'+job_id_core[0]+'.txt')

In [ ]:
videoHTML('IEI Tank Xeon (Intel Xeon CPU)',
          ['results/xeon/output.mp4'],
          'results/xeon/stats_'+job_id_xeon[0]+'.txt')

In [ ]:
videoHTML('IEI Intel GPU (Intel Core + Onboard GPU)', 
          ['results/gpu/output.mp4'],
          'results/gpu/stats_'+job_id_gpu[0]+'.txt')

In [ ]:
videoHTML('IEI Tank + IEI Mustang-F100-A10 (Intel® Arria® 10 FPGA)',
          ['results/fpga/output.mp4'],
          'results/fpga/stats_'+job_id_fpga[0]+'.txt')

In [ ]:
videoHTML('IEI Tank + Intel CPU + Intel NCS2',
          ['results/ncs2/output.mp4'],
          'results/ncs2/stats_'+job_id_ncs2[0]+'.txt')

### Assess Performance

In [ ]:
arch_list = [('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('xeon', 'Intel Xeon\nE3-1268L v5\nCPU'),
             ('gpu', ' Intel Core\ni5-6500TE\nGPU'),
             ('fpga', ' IEI Mustang\nF100-A10\nFPGA'),
             ('ncs2', 'Intel\nNCS2')]

stats_list = []
for arch, a_name in arch_list:
    if 'job_id_'+arch in vars():
        stats_list.append(('results/'+arch+'/stats_'+vars()['job_id_'+arch][0]+'.txt', a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))
summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time' )
summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps' )